# TP3: Bases de données, analyse et fouille

Ce troisième travail pratique vise à vous faire manipuler, explorer et interpréter des ensembles de données en transports. Vous êtes libres d’utiliser différents outils tels que
* des systèmes de gestion de base de données relationnel (SGBDR comme SQLite, PostgreSQL, Microsoft Access, etc.) ;
* des langages de programmation pour manipuler les données et les visualiser (Matlab, Python, R, etc.) ;
* des outils économétriques (Gretl, Tanagra, R, Python/StatsModels) ;
* des outils de fouille de données (Tanagra,Weka).

## 1. Jeu de données
Les données utilisées sont les temps de parcours collectés par la Ville de Montréal à l’aide de capteurs Bluetooth disponible sur le portail des données ouvertes. Les données sont collectées pour des paires origine-destination pré-définies, décrites dans le fichier des segments routiers de collecte des temps de parcours. Les données de temps de parcours disponibles couvrent la période du 1er janvier 2016 au 28 février 2017. Les attributs des deux fichiers de données sont décrits sur leurs pages respectives.

In [2]:
import pandas as pd
import numpy as np

In [4]:
seg_routier = pd.read_csv('./donnees/bornes.csv', sep=',', index_col = 0)
trips16 = pd.read_csv('./donnees/trips2016.csv', sep=',', index_col = 0)

## 2. Mandats
### 2.1 Modèle de données
Après analyse des fichiers des segments routiers et des temps de parcours, proposer un modèle de données relationnel pour ces données respectant les trois formes normales (retirer les données redondantes). Indiquer les clefs primaires, externes et les types de données des attributs.

Il n’est pas obligé d’utiliser ce modèle dans la suite des travaux.

In [5]:
seg_routier

,channel_name,active,LinkID,SrcDetectorId,SrcLatitude,SrcLongitude,DestDetectorId,DestLatitude,DestLongitude,LinkName,RouteDirectionName,SrcChannelId,DestChannelId,LineDistance_m,last_poll_time
IdLink,,,,,,,,,,,,,,,
1,Saint-Michel-Industriel,1,LCh_10-16,10,"45,5760283333333","-73,6364616666667",16,"45,564785","-73,6449433333333",Charland:Saint-Michel a Papineau,O,11,17,1410,2019-07-02 18:44:59
2,Papineau-charland,1,LCh_16-10,16,"45,564785","-73,6449433333333",10,"45,5760283333333","-73,6364616666667",Charland:Papineau a Saint-Michel,N,17,11,1414,2019-07-02 18:44:58
3,Papineau-Henri-Bourassa,1,LHB_15-20,15,"45,5702683333333","-73,6588066666667",20,"45,555365","-73,6683933333333",Henri-Bourassa:Papineau a Lajeunesse,O,16,21,1818,2019-07-02 18:44:57
4,Henri-Bourassa-Lajeunesse,1,LHB_20-15,20,"45,555365","-73,6683933333333",15,"45,5702683333333","-73,6588066666667",Henri-Bourassa:Lajeunesse a Papineau,N,21,16,1818,2019-02-15 13:56:37
5,Henri-Bourassa-Lajeunesse,1,LHB_20-21,20,"45,555365","-73,6683933333333",21,"45,5349866666667","-73,6807116666667",Henri_Bourassa:Lajeunesse a Bourassa-Acadie,O,21,22,2460,2019-02-15 13:56:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,Alexis-Nihon-Thimens,1,LTh_1O-1P,1O,"45,50951","-73,6975",1P,"45,5186","-73,70165",Thimens : Alexis-Nihon à Saint-Exupéry,N,137,138,1464,2019-07-02 18:44:58
366,Fullum-Notre-Dame,1,LRL_98-37,98,"45,52613","-73,54534",37,"45,5214166666667","-73,549805",René-Lévesque : Fullum à Papineau,O,120,49,629,2019-07-02 18:44:58
367,Notre-Dame-Letourneux,1,LND_1E-1M,1E,"45,55003","-73,53208",1M,"45,53643","-73,54156","Notre-Dame : Letourneux à Alphonse-D,-Roy",O,125,134,1683,2019-07-02 18:44:53


In [4]:
trips16

,SrcDetectorId,DestDetectorId,PathDistance_m,TripStart_dt,TripEnd_dt,Speed_kmh,TravelTime_s
LinkId,,,,,,,
LCC_35-34,35,34,448,2016-01-25 15:40:49,2016-01-25 15:41:16,"59,7333333",27
LCC_35-34,35,34,448,2016-01-25 15:40:52,2016-01-25 15:41:19,"59,7333333",27
LSh_05-06,5,6,403,2016-01-25 15:40:53,2016-01-25 15:41:21,"51,8142857",28
LSh_02-01,2,1,1192,2016-01-25 15:40:19,2016-01-25 15:41:41,"52,3317073",82
LSJ_41-39,41,39,298,2016-01-25 15:40:16,2016-01-25 15:41:45,"12,0539326",89
...,...,...,...,...,...,...,...
LLo_42-41,42,41,1115,2016-01-25 20:23:10,2016-01-25 20:33:00,"6,8033898",590
LSM_14-13,14,13,1753,2016-01-25 20:20:03,2016-01-25 20:33:03,"8,0907692",780
LPi_01-47,1,47,2570,2016-01-25 20:29:50,2016-01-25 20:33:04,"47,6907216",194


## 2.2 Analyse descriptive des temps de parcours
Pour cette première partie, il faut sélectionner deux segments routiers à étudier (le même segment dans les deux directions de déplacement, de A à B et de B vers A) de longueur supérieure à 1500 m. Indiquer clairement les deux segments sélectionnés.

In [6]:
seg_routier[seg_routier.LineDistance_m>1500]

,channel_name,active,LinkID,SrcDetectorId,SrcLatitude,SrcLongitude,DestDetectorId,DestLatitude,DestLongitude,LinkName,RouteDirectionName,SrcChannelId,DestChannelId,LineDistance_m,last_poll_time
IdLink,,,,,,,,,,,,,,,
3,Papineau-Henri-Bourassa,1,LHB_15-20,15,"45,5702683333333","-73,6588066666667",20,"45,555365","-73,6683933333333",Henri-Bourassa:Papineau a Lajeunesse,O,16,21,1818,2019-07-02 18:44:57
4,Henri-Bourassa-Lajeunesse,1,LHB_20-15,20,"45,555365","-73,6683933333333",15,"45,5702683333333","-73,6588066666667",Henri-Bourassa:Lajeunesse a Papineau,N,21,16,1818,2019-02-15 13:56:37
5,Henri-Bourassa-Lajeunesse,1,LHB_20-21,20,"45,555365","-73,6683933333333",21,"45,5349866666667","-73,6807116666667",Henri_Bourassa:Lajeunesse a Bourassa-Acadie,O,21,22,2460,2019-02-15 13:56:37
6,Henri-Bourassa-Acadie,1,LHB_21-20,21,"45,5349866666667","-73,6807116666667",20,"45,555365","-73,6683933333333",Henri_Bourassa:Acadie a Bourassa-Lajeunesse,E,22,21,2460,2019-07-02 18:45:02
11,Papineau-charland,1,LPa_16-17,16,"45,564785","-73,6449433333333",17,"45,5564283333333","-73,626345",Papineau:Charland a Barnabe-Larocque,S,17,18,1707,2019-07-02 18:44:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Cavendish-Poirier,1,LTh_1Q-1O,1Q,"45,50528","-73,71136",1O,"45,50951","-73,6975",Thimens : Poirier à Alexis-Nihon,E,139,137,1627,2019-07-02 18:44:56
361,Alexis-Nihon-Thimens,1,LTh_1O-1Q,1O,"45,50951","-73,6975",1Q,"45,50528","-73,71136",Thimens : Alexis-Nihon à Poirier,O,137,139,1564,2019-07-02 18:44:58
367,Notre-Dame-Letourneux,1,LND_1E-1M,1E,"45,55003","-73,53208",1M,"45,53643","-73,54156","Notre-Dame : Letourneux à Alphonse-D,-Roy",O,125,134,1683,2019-07-02 18:44:53


In [7]:
#Henri-Bourrassa de Papineau a Lajeunesse
seg_routier.iloc[2]

channel_name                       Papineau-Henri-Bourassa
active                                                   1
LinkID                                           LHB_15-20
SrcDetectorId                                           15
SrcLatitude                               45,5702683333333
SrcLongitude                             -73,6588066666667
DestDetectorId                                          20
DestLatitude                                     45,555365
DestLongitude                            -73,6683933333333
LinkName              Henri-Bourassa:Papineau a Lajeunesse
RouteDirectionName                                       O
SrcChannelId                                            16
DestChannelId                                           21
LineDistance_m                                        1818
last_poll_time                         2019-07-02 18:44:57
Name: 3, dtype: object

In [8]:
#Henri-Bourrassa de Lajeunesse a Papineau
seg_routier.iloc[3]

channel_name                     Henri-Bourassa-Lajeunesse
active                                                   1
LinkID                                           LHB_20-15
SrcDetectorId                                           20
SrcLatitude                                      45,555365
SrcLongitude                             -73,6683933333333
DestDetectorId                                          15
DestLatitude                              45,5702683333333
DestLongitude                            -73,6588066666667
LinkName              Henri-Bourassa:Lajeunesse a Papineau
RouteDirectionName                                       N
SrcChannelId                                            21
DestChannelId                                           16
LineDistance_m                                        1818
last_poll_time                         2019-02-15 13:56:37
Name: 4, dtype: object

### Étude de la distribution des temps de parcours
Décrire la distribution des temps de parcours dans une direction, par des statistiques descriptives et des figures. Proposer une méthode pour filtrer les temps de parcours les plus longs si nécessaire. Une fois les données filtrées, faire un test d’adéquation à la loi
normale (ou autre loi théorique plus adaptée).

Le cas échéant, la méthode de filtrage sera appliquée à toutes les données utilisées par
la suite.

In [9]:
trips16['TravelTime_s'].describe()

count    2.545524e+07
mean     2.831823e+02
std      3.650347e+02
min      6.000000e+00
25%      7.900000e+01
50%      1.590000e+02
75%      2.920000e+02
max      3.600000e+03
Name: TravelTime_s, dtype: float64